In [1]:
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import GlobalMaxPooling1D, Dense, Activation, Dropout, Embedding,Conv1D
import random


with open("/content/intents.json") as file:
    intents = json.load(file)

df = pd.DataFrame(columns=['Pattern', 'Tag'])

def extract_json_info(json_file, df):
    # Iterate over each intent in the JSON file
    for intent in json_file['intents']:
        # Iterate over each pattern in the current intent
        for pattern in intent['patterns']:
            # Create a list containing the pattern and its associated tag
            sentence_tag = [pattern, intent['tag']]
            # Append the pattern and tag to the DataFrame
            df.loc[len(df.index)] = sentence_tag
     # Return the updated DataFrame
    return df
df = extract_json_info(intents, df)
labels = df['Tag'].unique().tolist()
labels = [s.strip() for s in labels]
#count each tag
tag_counts = df['Tag'].value_counts()

# Tokenize the text patterns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Pattern'])

# Convert text patterns to numerical sequences
X = tokenizer.texts_to_sequences(df['Pattern'])

# Pad sequences to ensure uniform length
max_sequence_length = max(len(seq) for seq in X)
X_padded = pad_sequences(X, maxlen=max_sequence_length, padding='post')

# Convert tags to numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Tag'])


X_train, X_val, y_train, y_val = train_test_split(X_padded, y, test_size=0.2, random_state=42)

#Model Defination
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model to training data
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=32)



# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_accuracy:.4f}')

# Save the model
model.save('mymodel.h5')

Epoch 1/200
11/11 [==============================] - 1s 32ms/step - loss: 3.6258 - accuracy: 0.0187 - val_loss: 3.6115 - val_accuracy: 0.0247
Epoch 2/200
11/11 [==============================] - 0s 11ms/step - loss: 3.5910 - accuracy: 0.0685 - val_loss: 3.6033 - val_accuracy: 0.0123
Epoch 3/200
11/11 [==============================] - 0s 10ms/step - loss: 3.5427 - accuracy: 0.1121 - val_loss: 3.5902 - val_accuracy: 0.0247
Epoch 4/200
11/11 [==============================] - 0s 11ms/step - loss: 3.5041 - accuracy: 0.1090 - val_loss: 3.5730 - val_accuracy: 0.0494
Epoch 5/200
11/11 [==============================] - 0s 10ms/step - loss: 3.4608 - accuracy: 0.1153 - val_loss: 3.5557 - val_accuracy: 0.0370
Epoch 6/200
11/11 [==============================] - 0s 10ms/step - loss: 3.4359 - accuracy: 0.1121 - val_loss: 3.5307 - val_accuracy: 0.0494
Epoch 7/200
11/11 [==============================] - 0s 9ms/step - loss: 3.3757 - accuracy: 0.1340 - val_loss: 3.5023 - val_accuracy: 0.0494
Epoch 8

In [2]:
model.save("mymodel.h5")

In [3]:
from keras.models import load_model

# Loading the saved model
loaded_model = load_model('mymodel.h5')

# Function to preprocess input sentence
def preprocess_input_sentence(sentence, tokenizer, max_sequence_length):
    # Tokenize input sentence
    input_sequence = tokenizer.texts_to_sequences([sentence])
    # Pad sequences
    padded_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
    return padded_sequence

# Function to get response
def get_response(sentence, loaded_model, tokenizer, max_sequence_length, label_encoder):
    # Preprocess input sentence
    preprocessed_input = preprocess_input_sentence(sentence, tokenizer, max_sequence_length)
    # Predict label
    predicted_label = loaded_model.predict(preprocessed_input).argmax(axis=-1)
    # Convert label to tag
    predicted_tag = label_encoder.inverse_transform(predicted_label)
    return predicted_tag

#example
sentence = "who are u"
# Process the input sentence and get response
response = get_response(sentence, loaded_model, tokenizer, max_sequence_length, label_encoder)

# Print the response
print("Response:", response)

1/1 [==============================] - 0s 118ms/step
Response: ['task']


In [4]:
# Load the saved model
loaded_model = load_model('mymodel.h5')

# Function to preprocess input sentence
def preprocess_input_sentence(sentence, tokenizer, max_sequence_length):
    # Tokenize input sentence
    input_sequence = tokenizer.texts_to_sequences([sentence])
    # Pad sequences
    padded_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
    return padded_sequence

# Function to generate response
def generate_response(sentence, loaded_model, tokenizer, max_sequence_length, label_encoder):
    # Preprocess input sentence
    preprocessed_input = preprocess_input_sentence(sentence, tokenizer, max_sequence_length)
    # Predict probabilities for all classes
    predicted_probabilities = loaded_model.predict(preprocessed_input)
    # Get the index of the class with the highest probability
    predicted_label_index = np.argmax(predicted_probabilities, axis=-1)
    # Get the predicted tag based on the index
    predicted_tag = label_encoder.inverse_transform(predicted_label_index)
    return predicted_tag

# Example of conversation loop
print("Chatbot: Hello! How can I assist you today? (Type 'exit' to end)")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye! Have a great day!")
        break
    # Get response from the chatbot
    response = generate_response(user_input, loaded_model, tokenizer, max_sequence_length, label_encoder)
    print("Chatbot:", response[0])

Chatbot: Hello! How can I assist you today? (Type 'exit' to end)
You: hi
1/1 [==============================] - 0s 76ms/step
Chatbot: greeting
You:  bye
1/1 [==============================] - 0s 20ms/step
Chatbot: goodbye
You: exit
Chatbot: Goodbye! Have a great day!


In [5]:
# Load the model
loaded_model = load_model('mymodel.h5')

# Function to preprocess input sentence
def preprocess_input_sentence(sentence, tokenizer, max_sequence_length):
    # Tokenize input sentence
    input_sequence = tokenizer.texts_to_sequences([sentence])
    # Pad sequences
    padded_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
    return padded_sequence

# Function to get response
def get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents, df):
    # Preprocess input sentence
    preprocessed_input = preprocess_input_sentence(sentence, tokenizer, max_sequence_length)
    # Predict label
    predicted_label = loaded_model.predict(preprocessed_input).argmax(axis=-1)
    # Convert label to tag
    predicted_tag = label_encoder.inverse_transform(predicted_label)
    # Get the response based on the predicted tag
    if predicted_tag[0] in df['Tag'].values:
        response_row = df[df['Tag'] == predicted_tag[0]].iloc[0]
        response = response_row['Tag'] + ": " + df['Pattern']
    else:
        response = intents.get(predicted_tag[0], "I'm sorry, I didn't understand that.")
    return response

# Example usage
sentence = "Hi"
response = get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents, df)
print("Chatbot:", response)

1/1 [==============================] - 0s 82ms/step
Chatbot: 0                     greeting: Hi
1           greeting: How are you?
2       greeting: Is anyone there?
3                  greeting: Hello
4               greeting: Good day
                  ...             
397      greeting: ragging history
398    greeting: ragging incidents
399                  greeting: hod
400             greeting: hod name
401       greeting: who is the hod
Name: Pattern, Length: 402, dtype: object


In [6]:
# Function to preprocess input sentence
def preprocess_input_sentence(sentence, tokenizer, max_sequence_length):
    # Tokenize input sentence
    input_sequence = tokenizer.texts_to_sequences([sentence])
    # Pad sequences
    padded_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
    return padded_sequence

# Function to get response
def get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents):
    # Preprocess input sentence
    preprocessed_input = preprocess_input_sentence(sentence, tokenizer, max_sequence_length)
    # Predict label
    predicted_label = loaded_model.predict(preprocessed_input).argmax(axis=-1)
    # Convert label to tag
    predicted_tag = label_encoder.inverse_transform(predicted_label)

    # Iterate through the intents to find the matching tag
    for intent in intents["intents"]:
        if intent["tag"] == predicted_tag[0]:
            # Select a random response from the list of responses
            response = random.choice(intent["responses"])
            return response

    # If no matching intent is found, return a default message
    return "I'm sorry, I didn't understand that."


print("Chatbot: Hello! How can I assist you today? (Type 'exit' to end)")
while True:
    # Get user input
    sentence = input("You: ")
    # Check if the user wants to exit
    if sentence.lower() == 'exit':
        print("Chatbot: Goodbye! Have a great day!")
        break
    # Get response from chatbot
    response = get_response(sentence, loaded_model, tokenizer, max_sequence_length, intents)
    print("Chatbot:", response)

Chatbot: Hello! How can I assist you today? (Type 'exit' to end)
You: goodbye
1/1 [==============================] - 0s 22ms/step
Chatbot: Come back soon
You: exit
Chatbot: Goodbye! Have a great day!
